# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:
LaborStatisticsDB
    dbo.annual_2016:
    id/ series_id/ year/ period/ footnote_code/ original_file

    dbo.datatype:
    data_type_code/ data_type_text (categories of types of workers/hourly, weekly pay, employmenht for women, etc.)

    dbo.footnote:
    footnote_code/ footnote_text (seasonally adjusted, priliminary)

    dbo.industry:
    id/ industry_code/ naics_code/ publishing_status/ industry_name/ display_level/ selectable/ sort_sequence

    dbo.january_2017:
    id/ series_id/ year/ period/ value/ footnote_code/ original_file 

    dbo.period:
    period_code/ month_abbr/ month 

    dbo.seasonal: 
    industry_code/ seasonal_text (seasonally adjusted, not seasonally adjusted)

    dbo.series:
    series_id/ supersector_code/ industry_code/ data_type_code/ seasonal/ series_title

    dbo.supersector:
    supersector_code/ supersector_name 


    Connections: 
    ID: dbo.annual_2016, dbo.industry, dbo.january_2017, 

    Series_ID: dbo.annual_2016, dbo.january_2017, dbo.series_id

    Period: dbo.period, dbo.annual_2016, dbo.january_2017

    Footnote code: dbo.footnote, dbo.annual_2016, dbo.january_2017

    Original File: dbo.annual_2016, dbo.january_2017

    Industry Code: dbo.industry, dbo.seasonal, dbo.series 

    Supersector code: dbo.supersector, dbo.series, 

    Data_type_code: dbo.datatype, dbo.series
    


2. What is the datatype for women employees?

In [ ]:
select *
from LaborStatisticsDB.dbo.datatype 
where data_type_text like '%women%'
-- data_type_code 10: Women Employees
--data_type_code 39: Women Employees-to-all employees ratio 

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
select *
from LaborStatisticsDB.dbo.series
where industry_code = 55522110 and series_title like '%women%' and supersector_code = 55
--series ID CES5552211010

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
select sum (value)
from laborstatisticsdb.dbo.annual_2016 as A
left join laborstatisticsdb.dbo.series as S 
 ON a.series_id = s.series_id
 where data_type_code in (1, 25, 26)
--shows value as 2340612

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:
SELECT SUM(a.value) AS total_value
FROM LaborStatisticsDB.dbo.annual_2016 AS a
LEFT JOIN LaborStatisticsDB.dbo.series AS s
  ON a.series_id = s.series_id
WHERE s.data_type_code = 10
--1125490 women employees in all industries 

3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:
SELECT SUM(a.value) AS total_value
FROM LaborStatisticsDB.dbo.annual_2016 AS a
LEFT JOIN LaborStatisticsDB.dbo.series AS s
  ON a.series_id = s.series_id
WHERE s.data_type_code = 6
--production/ non-supervisory employees reported were 1263650


4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
select *
from LaborStatisticsDB.dbo.series
where series_title like '%production%'
--data type code for average weekly hours worked by production and nonsupervisory employees= 7

SELECT SUM(j.value) AS total_value
FROM LaborStatisticsDB.dbo.january_2017 AS j
LEFT JOIN LaborStatisticsDB.dbo.series AS s
  ON j.series_id = s.series_id
WHERE s.data_type_code = 7
--average weekly hours worked by production and non-supervisory employees in january 2017= 79473.2

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:

select sum(value)
from LaborStatisticsDB.dbo.january_2017
where series_id in (
  select series_id
  from laborstatisticsdb.dbo.series
  where series_title like 'aggregate weekly payrolls of production and nonsupervisory employees'
  )
  -- aggregate weekly payroll for production and nonsupervisory employees january-2017 was 1,838,753,220

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
select i.industry_name, j.value
from LaborStatisticsDB.dbo.series s 
inner join LaborStatisticsDB.dbo.january_2017 j
  on s.series_id = j.series_id
inner join LaborStatisticsDB.dbo.industry i
on i.industry_code = s.industry_code 
where s.series_title = 'average weekly hours of production and nonsupervisory employees'
order by j.value DESC
--industry with highest average hours in January was Motor Vehicle Power Train Components- 49.8 hours 

select i.industry_name, j.value
from LaborStatisticsDB.dbo.series s 
inner join LaborStatisticsDB.dbo.january_2017 j
  on s.series_id = j.series_id
inner join LaborStatisticsDB.dbo.industry i
on i.industry_code = s.industry_code 
where s.series_title = 'average weekly hours of production and nonsupervisory employees'
order by j.value asc 
--industry with lowest average hours in january was Fitness and recreational sports centers --16.7 hours

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
select i.industry_name, j.value
from LaborStatisticsDB.dbo.series s 
inner join LaborStatisticsDB.dbo.january_2017 j
  on s.series_id = j.series_id
inner join LaborStatisticsDB.dbo.industry i
on i.industry_code = s.industry_code 
where s.series_title = 'aggregate weekly payrolls of production and nonsupervisory employees'
order by j.value DESC
--total private has highest weekly payrolls at 74,498,171

select i.industry_name, j.value
from LaborStatisticsDB.dbo.series s 
inner join LaborStatisticsDB.dbo.january_2017 j
  on s.series_id = j.series_id
inner join LaborStatisticsDB.dbo.industry i
on i.industry_code = s.industry_code 
where s.series_title = 'aggregate weekly payrolls of production and nonsupervisory employees'
order by j.value asc
--lowest weekly payrolls is for coin-operated laundries and drycleaners at 10,079

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *
select top 50 *
from laborstatisticsdb.dbo.annual_2016 as a
left join laborstatisticsdb.dbo.series as S 
on a.series_id = s.series_id 


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

select top 50 *
from laborstatisticsdb.dbo.annual_2016 as a
left join laborstatisticsdb.dbo.series as S 
on a.series_id = s.series_id 
order by id 

--used a left join to keep the annual_2016 table 

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *
select top 50*
from laborstatisticsdb.dbo.series as series 
left join laborstatisticsdb.dbo.datatype as datatype 
on series.data_type_code = datatype.data_type_code 

-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id
select top 50*
from laborstatisticsdb.dbo.series as series 
left join laborstatisticsdb.dbo.datatype as datatype 
on series.data_type_code = datatype.data_type_code 
order by series_id 

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *
select top 50* 
from laborstatisticsdb.dbo.series as S  
left join laborstatisticsdb.dbo.industry as i 
on s.industry_code = i.industry_code 

-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id
select top 50* 
from laborstatisticsdb.dbo.series as S  
left join laborstatisticsdb.dbo.industry as i 
on s.industry_code = i.industry_code 
order by series_id

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:

select j.series_id, i.industry_code, i.industry_name, j.value
from LaborStatisticsDB.dbo.january_2017 as j 
inner join LaborStatisticsDB.dbo.series as s 
on j.series_id = s.series_id 
left join LaborStatisticsDB.dbo.industry as i  
on s.industry_code = i.industry_code
where j.value >
(select avg(value)
from laborstatisticsdb.dbo.annual_2016 as annual
left join laborstatisticsdb.dbo.series as series 
on annual.series_id = series.series_id 
where data_type_code = 82
)
order by value ASC 
--average aggregate weekly payrolls of production and nonsupervisory employees in 2016: 819960.34
--results do not show production and nonsupervisory employees with amassing the same weekly payrolls in january 2017

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- Optional CTE below

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
select avg(value)
from laborstatisticsdb.dbo.annual_2016 as annual
left join laborstatisticsdb.dbo.series as series 
on annual.series_id = series.series_id
where data_type_code =30 
union (
  select avg(value)
  from laborstatisticsdb.dbo.january_2017 as january
  left join LaborStatisticsDB.dbo.series as series 
  on january.series_id = series.series_id
  where data_type_code = 30
)
--weekly average payrolls for production and nonsupervisory employees:
--annual_2016: 797.20
--january_2017: 808.53

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

    Based on the data, production and nonsupervisory employees earned more money on average in january 2017 compared to their averages in 2016. Average weekly payrolls for this group in 2016 was $797.20 and in January 2017, $808.53.  Production and nonsupervisory employees were better off in 2017 since they were paid more. 

2. In which industries did production and nonsupervisory employees fare better?

select avg(annual.value) as average_weekly_earnings, data_type_code, industry_name, series_title
from laborstatisticsDB.dbo.series as series
inner join laborstatisticsDB.dbo.industry as industry
on series.industry_code = industry.industry_code
inner join laborstatisticsdb.dbo.annual_2016 as annual
on annual.series_id = series.series_id  
where series_title like '%average weekly earnings of production and nonsupervisory employees%'
group by data_type_code, industry.industry_name, series.series_title 
order by average_weekly_earnings DESC   



Based on the query above, the top 5 industries wit the highest average weekly earnings are:
1.) Pipeline Transportation ($1730.96)
2.) Software Publishers ($1720.42)
3.) Petroleum and Coal Products ($1718.26)
4.) Fossil Fuel Electric Power Generation ($1712.65)
5.) Reinsurance Carriers ($1699.97)



select avg(annual.value) as average_weekly_hours, data_type_code, industry_name, series_title
from laborstatisticsDB.dbo.series as series
inner join laborstatisticsDB.dbo.industry as industry
on series.industry_code = industry.industry_code
inner join laborstatisticsdb.dbo.annual_2016 as annual
on annual.series_id = series.series_id  
where series_title like '%average weekly hours of production and nonsupervisory employees%'
group by data_type_code, industry.industry_name, series.series_title 
order by average_weekly_hours DESC   



Based on the query above, the top 5 industries with the highest average weekly hours are:
1.) Motor Vehicle Power Train Components (51.4 hours)
2.) Crushed and Broken Limestone Mining (48.4)
3.) Oil and Gas Pipeline Construction (47.8)
4.) Stone Mining and Quarrying (47.6)
5.) Support Activities for Mining (46.7)


select avg(annual.value) as average_hourly_earnings, data_type_code, industry_name, series_title
from laborstatisticsDB.dbo.series as series
inner join laborstatisticsDB.dbo.industry as industry
on series.industry_code = industry.industry_code
inner join laborstatisticsdb.dbo.annual_2016 as annual
on annual.series_id = series.series_id  
where series_title like '%average hourly earnings of production and nonsupervisory employees%'
group by data_type_code, industry.industry_name, series.series_title 
order by average_hourly_earnings DESC  



--Based on the above query, the industries with the highest average hourly earnings are:
1.) Reinsurance Carriers ($46.63/hour)
2.) Software Publishers ($45.34/hour) 
3.) Portfolio Management ($43.64/hour) 
4.) Customer Computer Programming Services ($42.50/hour) 
5.) Computer Systems Design and Related Services ($41.29)




My analysis shows that more industrial industries have most opportunies for working extra hours, thus likely making more money. I found that the industries that have a higher rate of pay are tend to be employees that are in computer sciences, finances, and insurance.  


3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

Going further with the analysis, I think it would be intesting to have data on how many women work in each industry. For instance, I found in my analyses that more industrial industries that are focused in quarrying rocks and manufacturing mechanical components have the highest number of hours worked, on average. I am curious what percentage of the workforce for those jobs are held by women. Furthermore, what percentage of women hold positions in the highest paying industries such as software publishing, portfolio management and computer sciences?  Do women tend to make less money than men because they hold jobs that statistically pay less per hour? How much do women get in the highest paid industries compared to the average? What is the difference between average male and female wages in the same industry? 

These are topics that I think would be interesting to figure out if there were more data points to compare within industries. 